## Music Recommendation System (Machine Learning)

This project is aimed upon building a music recommendation system that gives the user recommendations on music based on his music taste by analysing his previously heard music and playlist. This project is done in two ways, using 'User - to - User Recommendation' and 'Item - to - Item Recommendation'. Birch, MiniBatchKMeans and KMeans algorithms are being used along with 'Surprise' module to compute the similarity between recommendations and user's already existing playlist for evaluation

### Obtaining Data

In [836]:
import pandas as pd
import numpy as np

In [837]:
final = pd.read_csv('datasets/final/main_data.csv')
metadata = pd.read_csv('datasets/final/metadata.csv')

### Model Selection - K Means Algorithm

In [838]:
from sklearn.cluster import KMeans
from sklearn.utils import shuffle

In [839]:
from sklearn.preprocessing import LabelEncoder
Weather_encoder=LabelEncoder()
final["Weather"]=Weather_encoder.fit_transform(final["Weather"])
Motion_encoder=LabelEncoder()
final["Motion"]=Motion_encoder.fit_transform(final["Motion"])
Location_encoder=LabelEncoder()
final["Location"]=Location_encoder.fit_transform(final["Location"])

In [840]:
final = shuffle(final)

In [841]:
X = final.loc[[i for i in range(0, 500)]]
Y = final.loc[[i for i in range(500, final.shape[0])]]

In [842]:
X = shuffle(X)
Y = shuffle(Y)

In [843]:
metadat = metadata[0:1000]

In [844]:
metadata.head()

,track_id,album_title,artist_name,genre,track_title
0,2,AWOL - A Way Of Life,AWOL,HipHop,Food
1,3,AWOL - A Way Of Life,AWOL,HipHop,Electric Ave
2,5,AWOL - A Way Of Life,AWOL,HipHop,This World
3,10,Constant Hitmaker,Kurt Vile,Pop,Freeway
4,134,AWOL - A Way Of Life,AWOL,HipHop,Street Music


In [845]:
metadata = metadata.set_index('track_id')

In [846]:
kmeans = KMeans(n_clusters=6)

In [847]:
Y.head()

,Unnamed: 0,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,Weather,Motion,Location
646,646,1599,0.000013,0.433420,0.994968,0.867411,0.127507,0.061677,146.139,0.110264,5,2,5
813,813,1829,0.739425,0.535139,0.458887,0.920171,0.096835,0.481509,160.027,0.772956,0,3,7
749,749,1750,0.802331,0.403832,0.639454,0.005084,0.098483,0.030431,116.052,0.234074,6,2,3
541,541,1354,0.211803,0.374616,0.976179,0.883576,0.148260,0.179517,85.050,0.188398,1,3,8
956,956,3360,0.644569,0.510711,0.447602,0.627566,0.107032,0.042808,175.453,0.933311,3,1,7


In [848]:
def fit(df, algo, flag=0):
    if flag:
        algo.fit(df)
    else:
         algo.partial_fit(df)          
    df['label'] = algo.labels_
    return (df, algo)

In [849]:
def predict(t, Y):
    y_pred = t[1].predict(Y)
    mode = pd.Series(y_pred).mode()
    return t[0][t[0]['label'] == mode.loc[0]]

In [850]:
def recommend(recommendations, meta, Y):
    dat = []
    for i in Y['track_id']:
        dat.append(i)
    genre_mode = meta.loc[dat]['genre'].mode()
    artist_mode = meta.loc[dat]['artist_name'].mode()
    return meta[meta['genre'] == genre_mode.iloc[0]], meta[meta['artist_name'] == artist_mode.iloc[0]], meta.loc[recommendations['track_id']]

In [851]:
t = fit(X, kmeans, 1)

In [852]:
recommendations = predict(t, Y)

In [853]:
output = recommend(recommendations, metadata, Y)

In [854]:
genre_recommend, artist_name_recommend, mixed_recommend = output[0], output[1], output[2]

In [855]:
genre_recommend.shape

(3892, 4)

In [856]:
artist_name_recommend.shape

(65, 4)

In [857]:
mixed_recommend.shape

(74, 4)

In [858]:
# Genre wise recommendations
genre_recommend.head()

,album_title,artist_name,genre,track_title
track_id,,,,
153,Arc and Sender,Arc and Sender,Rock,Hundred-Year Flood
154,Arc and Sender,Arc and Sender,Rock,Squares And Circles
155,unreleased demo,Arc and Sender,Rock,Maps of the Stars Homes
169,Boss of Goth,Argumentix,Rock,Boss of Goth
170,Nightmarcher,Argumentix,Rock,Industry Standard Massacre


In [859]:
# Artist wise recommendations
artist_name_recommend.head()

,album_title,artist_name,genre,track_title
track_id,,,,
777,Quickies,Glove Compartment,Rock,Relaxing
778,Quickies,Glove Compartment,Rock,Don't Call Me Tonite / Very Good
779,Quickies,Glove Compartment,Rock,Causing Disturbances
780,Quickies,Glove Compartment,Rock,Don't Peek-O
783,Quickies,Glove Compartment,Rock,Puffy Onyx


In [860]:
# Mixed Recommendations
mixed_recommend.head()

,album_title,artist_name,genre,track_title
track_id,,,,
1170,Dead And/Or Famous,Mors Ontologica,Rock,Bad For Business
1280,John,Newagehillbilly,Rock,history
1169,Dead And/Or Famous,Mors Ontologica,Rock,Uneasy
1274,Rehersal,Necropolis,Rock,Stumpf
1190,Don't Cry,Mors Ontologica,Rock,Up


In [861]:
recommendations.head()

,Unnamed: 0,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence,Weather,Motion,Location,label
456,456,1170,0.698553,0.527502,0.920360,0.925601,0.104572,0.065797,135.110,0.300625,1,2,4,4
493,493,1280,0.094643,0.188684,0.769396,0.942995,0.105225,0.041025,139.145,0.039909,5,1,4,4
455,455,1169,0.567768,0.683491,0.922510,0.875908,0.077960,0.222589,128.063,0.925308,6,3,1,4
490,490,1274,0.215794,0.308622,0.999553,0.884130,0.166080,0.058445,120.367,0.032084,1,2,0,4
472,472,1190,0.396481,0.617996,0.949502,0.934739,0.084356,0.041216,100.068,0.909199,5,2,3,4


In [862]:
artist_name_recommend['artist_name'].value_counts()

Glove Compartment    65
Name: artist_name, dtype: int64

In [863]:
genre_recommend['genre'].value_counts()

Rock    3892
Name: genre, dtype: int64

In [864]:
genre_recommend['artist_name'].value_counts()

Glove Compartment               65
Blah Blah Blah                  62
Mors Ontologica                 50
Les Baudouins Morts             38
Kraus                           35
                                ..
Alone in 1982                    1
Ostrich Tuning                   1
The Dalai Lama Rama Fa Fa Fa     1
The Rusty Bells                  1
Lost Boy                         1
Name: artist_name, Length: 725, dtype: int64

#### Testing

In [865]:
testing = Y.iloc[6:12]['track_id']

In [866]:
testing

780    1794
684    1659
756    1759
909    2073
865    1919
889    1954
Name: track_id, dtype: int64

In [867]:
ids = testing.loc[testing.index]

In [868]:
songs = metadata.loc[testing.loc[list(testing.index)]]

In [869]:
songs

,album_title,artist_name,genre,track_title
track_id,,,,
1794,The Glove Compartment,Glove Compartment,Rock,Wrinkle Star
1659,Wackity Schmackity Doo,So Cow,Pop,Arse Above Tit
1759,Meet We the Medicine,The Agrarians,Folk,The Mirror Travels Above The Ocean
2073,Mono M::P Free,Thomas Dimuzio,Electronic,Skullshop
1919,Pioneers Toasted - Rare And Unreleased 2001-2004,The Twin Atlas,Folk,Plains Of Saviors
1954,Von Hayes,Von Hayes,Rock,Chairman of the Ask Me Society


In [870]:
re = predict(t, Y.iloc[6:12])

In [871]:
output = recommend(re, metadata, Y.iloc[6:12])

In [872]:
ge_re, ge_ar, ge_mix = output[0], output[1], output[2]

In [873]:
ge_re.head()

,album_title,artist_name,genre,track_title
track_id,,,,
139,Every Man For Himself,Alec K. Redfearn & the Eyesores,Folk,CandyAss
140,The Blind Spot,Alec K. Redfearn & the Eyesores,Folk,Queen Of The Wires
141,Every Man For Himself,Alec K. Redfearn & the Eyesores,Folk,Ohio
142,The Quiet Room,Alec K. Redfearn & the Eyesores,Folk,Punjabi Watery Grave
189,What I Know,Ed Askew,Folk,The Thought Of It


In [874]:
ge_ar.head(10)

,album_title,artist_name,genre,track_title
track_id,,,,
777,Quickies,Glove Compartment,Rock,Relaxing
778,Quickies,Glove Compartment,Rock,Don't Call Me Tonite / Very Good
779,Quickies,Glove Compartment,Rock,Causing Disturbances
780,Quickies,Glove Compartment,Rock,Don't Peek-O
783,Quickies,Glove Compartment,Rock,Puffy Onyx
786,Quickies,Glove Compartment,Rock,Electric Guru
787,Quickies,Glove Compartment,Rock,Booger-Flicking Winners Go Freeh
788,Quickies,Glove Compartment,Rock,Vert
790,Quickies,Glove Compartment,Rock,"Up, Two?"


In [875]:
ge_mix.head(10)

,album_title,artist_name,genre,track_title
track_id,,,,
1170,Dead And/Or Famous,Mors Ontologica,Rock,Bad For Business
1280,John,Newagehillbilly,Rock,history
1169,Dead And/Or Famous,Mors Ontologica,Rock,Uneasy
1274,Rehersal,Necropolis,Rock,Stumpf
1190,Don't Cry,Mors Ontologica,Rock,Up
1183,The Used Kids Session,Mors Ontologica,Rock,Vampires In My Neighborhood
1167,Dead And/Or Famous,Mors Ontologica,Rock,Lazy Suicide
1273,Demo,Necropolis,Rock,"No Tongue, No Talking"
1141,Since Before Inertia,Minmae,Rock,New Lilith/Flashing Timex


In [876]:
ge_re.shape

(874, 4)

In [877]:
ge_ar.shape

(65, 4)

In [878]:
ge_mix.shape

(74, 4)